In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [2]:
data = pd.read_csv("C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\2017-2022copy3.csv", parse_dates=['Tanggal'], date_format='%m/%d/%Y')

In [3]:
checkpoint = ModelCheckpoint('C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\checkpoints\\model_2.h5', monitor='val_loss', verbose=1, save_best_only=True)

In [4]:
tanggal = data["Tanggal"]
latitude = data["Latitude"]
longitude = data["Longitude"]
kedalaman = data["Kedalaman (km)"]
magnitude = data["Magnitude"]

In [5]:
gempa_semua = []
for index in range(len(latitude)):
  gempa = []
  gempa.append(tanggal[index])
  gempa.append(latitude[index])
  gempa.append(longitude[index])
  gempa.append(kedalaman[index])

  gempa_semua.append(gempa)

In [6]:
longitude.dtype

dtype('float64')

In [7]:
print(gempa_semua)

[[Timestamp('2017-01-01 00:00:00'), -8.93, 110.05, 10], [Timestamp('2017-01-02 00:00:00'), -8.94, 108.75, 10], [Timestamp('2017-01-03 00:00:00'), -8.39, 110.69, 10], [Timestamp('2017-01-04 00:00:00'), -9.27, 113.48, 10], [Timestamp('2017-01-05 00:00:00'), -9.81, 111.75, 10], [Timestamp('2017-01-06 00:00:00'), -8.69, 111.85, 26], [Timestamp('2017-01-07 00:00:00'), -3.0, 115.5346426, 10], [Timestamp('2017-01-08 00:00:00'), -9.12, 112.19, 10], [Timestamp('2017-01-09 00:00:00'), -8.19, 108.57, 73], [Timestamp('2017-01-10 00:00:00'), -7.1, 107.4757765, 12], [Timestamp('2017-01-11 00:00:00'), -7.98, 110.42, 3], [Timestamp('2017-01-12 00:00:00'), -8.41, 109.13, 23], [Timestamp('2017-01-13 00:00:00'), -7.99, 110.41, 10], [Timestamp('2017-01-14 00:00:00'), -8.63, 110.45, 25], [Timestamp('2017-01-15 00:00:00'), -7.35, 110.43, 2], [Timestamp('2017-01-16 00:00:00'), -9.31, 112.3, 10], [Timestamp('2017-01-17 00:00:00'), -8.87, 111.09, 10], [Timestamp('2017-01-18 00:00:00'), -9.01, 110.2, 20], [Time

In [8]:
X = (gempa_semua)
y = (magnitude)

In [9]:
X_df = pd.DataFrame(X, columns=['tanggal', 'latitude', 'longitude', 'kedalaman'])

In [10]:
X_df['tanggal'] = pd.to_numeric(X_df['tanggal'])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

NameError: name 'X_scaled' is not defined

In [ ]:
X_training, X_valid, y_training, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_training = np.reshape(X_training, (X_training.shape[0], 1, X_training.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
X_valid = np.reshape(X_valid, (X_valid.shape[0], 1, X_valid.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(1, X_training.shape[2])))
model.add(Dense(units=1, activation='linear'))
model.summary()

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
history = model.fit(X_training, y_training, batch_size=64, epochs=1000, verbose=1, validation_data=(X_valid, y_valid), callbacks=[checkpoint])

In [ ]:
model = load_model("C:\\Users\\wajit\\Documents\\GitHub\\Project-ANN\\checkpoints\\model_2.h5")
loss = model.evaluate(X_valid, y_valid)
print("Loss:", loss)

In [ ]:
new_latitude = 0.0
new_longitude = 0.0
new_depth = 0.0
new_date = '2023-01-01'

new_data = np.array([[new_latitude, new_longitude, new_depth, pd.to_datetime(new_date).timestamp()]])
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = np.reshape(new_data_scaled, (new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

num_predictions = 100
predicted_magnitudes = []

for predicted_magnitudes in range(num_predictions):
    predicted_magnitude = model.predict(new_data_reshaped)
    predicted_magnitudes.append(predicted_magnitude)
    new_data_reshaped = np.concatenate((new_data_reshaped, predicted_magnitude), axis=1)

predicted_magnitudes = np.array(predicted_magnitudes)

print("Predicted Magnitudes:")
print(predicted_magnitudes)

In [ ]:
num_days = 365
start_date = pd.to_datetime(new_date)
dates = pd.date_range(start_date, periods=num_days, freq='D')
future_data = np.repeat(new_data_scaled, num_days, axis=0)
future_data[:, -1] = dates.astype(np.int64)
future_data_reshaped = np.reshape(future_data, (future_data.shape[0], 1, future_data.shape[1]))
predicted_magnitude_future = model.predict(future_data_reshaped)

In [ ]:
plt.plot(dates, magnitude[:365], label='original Magnitude')
plt.plot(dates, predicted_magnitude_future, label='Predicted Magnitude')
plt.xlabel('Date')
plt.ylabel('Magnitude')
plt.title('Magnitude Prediction for the Next Year')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('error')
plt.title('Perbandingan')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.legend(['Original', 'Forecast'], loc='upper right')
plt.show()